# 1. Import

In [1]:
#gym imports
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

# Helper imports
import numpy as np
import random
import os

# import stable baselines stuff
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Types of Spaces

In [2]:
# used for discrete number of actions but want only 1
Discrete(3).sample() 

2

In [3]:
# continuous variables, upper, lower, then shape
Box(0,1, shape=(3,)).sample()

array([0.6468273 , 0.98046225, 0.9102171 ], dtype=float32)

In [4]:
#currently not supported by gym
Tuple((Discrete(3), Box(0,1, shape=(3,)))).sample()

(2, array([0.4788525 , 0.50169337, 0.5556536 ], dtype=float32))

In [5]:
# different types of spaces pass through
Dict({'height': Discrete(2), 'speed': Box(0,100, shape=(1,))}).sample()

OrderedDict([('height', 0), ('speed', array([85.525375], dtype=float32))])

In [6]:
#could add to tuple, or dict, number of positions and only 0,1 result
MultiBinary(4).sample()

array([0, 0, 1, 1], dtype=int8)

In [7]:
#
MultiDiscrete([5,2,2]).sample()

array([0, 0, 1], dtype=int64)

# 3. Building an Env
- build an agent to give us the best shower possible
- random temp
- 37 and 39 degrees

In [8]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        # Reset shower time
        self.shower_length = 60 
        return self.state

In [9]:
env = ShowerEnv()

C:\Users\Kamuela\anaconda3\envs\RLin3hr\lib\site-packages\gym\logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


In [10]:
env.observation_space

Box([0.], [100.], (1,), float32)

In [11]:
env.reset()

array([39.])

In [12]:
from stable_baselines3.common.env_checker import check_env

In [13]:
check_env(env, warn=True)

AssertionError: The observation returned by the `reset()` method does not match the given observation space

In [108]:
episodes = 5
for episode in range(1, episodes + 1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode,score))
env.close()

Episode:1 Score:-20
Episode:2 Score:-60
Episode:3 Score:-40
Episode:4 Score:-42
Episode:5 Score:16


# 5. Train

In [109]:
log_path = os.path.join('Training', 'Logs')
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [110]:
model.learn(total_timesteps = 20000)

Logging to Training\Logs\PPO_9
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -37.6    |
| time/              |          |
|    fps             | 462      |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 60         |
|    ep_rew_mean          | -37.2      |
| time/                   |            |
|    fps                  | 611        |
|    iterations           | 2          |
|    time_elapsed         | 6          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.00823976 |
|    clip_fraction        | 0.0777     |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.09      |
|    explained_variance   | 0.000191   |
|    learning_

# 6. Save model

In [ ]:
ppo_path = os.path.join('Training', 'Saved Models', 'Shower_Model_PPO')

In [ ]:
model.save(ppo_path)

In [ ]:
del model


In [ ]:
model = PPO.load(ppo_path, env)

In [14]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

NameError: name 'model' is not defined